In [1]:
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as Ker

**I Simulation des chemins**

In [2]:

def pay_off(S, r, K, dt):
    return tf.maximum(0.0, K - S)

def simulate_paths_(M, T, n, r, vol, S_initial, K, dt):
    nudt = (r - 0.5 * vol**2) * dt
    lnS = tf.math.log(S_initial)  # Log of the initial stock price

    # Generate normal random variables for the Monte Carlo simulation
    random =  tf.random.normal([M, 2*n]) #to be sure that they ll be independent
    Lw = tf.sqrt(0.5*dt)*random[:, n:2*n]
    dW = tf.sqrt(dt) * random[:,:n]

    # Compute the increments using the generated random variables
    delta_lnSt = nudt + vol * dW

    # Initialize and fill in the log prices
    LnS_s = tf.TensorArray(dtype=tf.float32, size=n+1, dynamic_size=False, clear_after_read=False)
    LnS_s = LnS_s.write(0, tf.fill([M], lnS))

    for j in range(1, n + 1):
        previous_values = LnS_s.read(j - 1)
        current_values = previous_values + delta_lnSt[:, j-1]
        LnS_s = LnS_s.write(j, current_values)

    # Convert log prices back to normal prices
    final_LnS_s = LnS_s.stack()
    S = tf.exp(tf.transpose(final_LnS_s))
    return S, dW , Lw

# Parameters
M = 100000  # Number of simulations
n = 50  # Number of steps
T = 1
dt = T / n
r = 0.06  # Risk-free rate
vol = 0.2  # Volatility
S_initial = 36.0  # Initial stock price
K=40.0
# Run the simulation
S, dW , Lw = simulate_paths_(M, T, n, r, vol, S_initial, K, dt)

In [3]:
X = tf.Variable(tf.zeros([M, n+1], dtype=tf.float32))
Y = tf.Variable(tf.zeros([M, n+1], dtype=tf.float32))
X_last_col = pay_off(S[:, n], r, K, dt)
X[:, n].assign(X_last_col)
Y[:, n].assign(X_last_col)
beta_dt = tf.exp(-r * dt)

**II Construction et entrainement du modèle**

In [4]:


with tf.device('/GPU:0'):
    inputs = tf.keras.Input(shape=(1,))


    x1 = layers.Dense(100, activation="relu")(inputs)
    x1 = layers.Dense(100, activation="relu")(x1)
    x1 = layers.Dense(100, activation="relu")(x1)
    x1 = layers.Dense(1)(x1)

    x2 = layers.Dense(100, activation="relu")(inputs)
    x2 = layers.Dense(100, activation="relu")(x2)
    x2 = layers.Dense(100, activation="relu")(x2)
    x2 = layers.Dense(100, activation="relu")(x2)
    x2 = layers.Dense(1)(x2)

    x3 = layers.Dense(100, activation="relu")(inputs)
    x3 = layers.Dense(100, activation="relu")(x3)
    x3 = layers.Dense(100, activation="relu")(x3)
    x3 = layers.Dense(100, activation="relu")(x3)
    x3 = layers.Dense(100, activation="relu")(x3)
    x3 = layers.Dense(1)(x3)


    x4 = layers.Dense(100, activation="relu")(inputs)
    x4 = layers.Dense(100, activation="relu")(x4)
    x4 = layers.Dense(100, activation="relu")(x4)
    x4 = layers.Dense(100, activation="relu")(x4)
    x4 = layers.Dense(100, activation="relu")(x4)
    x4 = layers.Dense(100, activation="relu")(x4)
    x4 = layers.Dense(1)(x4)

    x5 = layers.Dense(100, activation="relu")(inputs)
    x5 = layers.Dense(100, activation="relu")(x5)
    x5 = layers.Dense(100, activation="relu")(x5)
    x5 = layers.Dense(100, activation="relu")(x5)
    x5 = layers.Dense(100, activation="relu")(x5)
    x5 = layers.Dense(100, activation="relu")(x5)
    x5 = layers.Dense(100, activation="relu")(x5)
    x5 = layers.Dense(1)(x5)


    outputs = layers.concatenate([x1, x2, x3 , x4, x5])


    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="three_param_model")

    model.summary()

    def custom_loss(y_true, y_pred):
        Y_true = y_true[:, 0]
        dW_true = y_true[:, 1]
        dW_true_2 = tf.square(dW_true)
        term_2 =  dW_true_2 - dt
        term_4 = dW_true*((1/3)*dW_true_2 - dt)

        squared_difference = tf.square(Y_true - (y_pred[:, 0] + dW_true * y_pred[:, 1] + term_2 * y_pred[:, 2]+ y_true[ : , 2]*y_pred[ : , 3] + term_4*y_pred[ : , 4]) )
        custom_loss_value = tf.reduce_mean(squared_difference)

        return custom_loss_value

    model.compile(optimizer='adam', loss=custom_loss)


Model: "three_param_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 dense_22 (Dense)            (None, 100)                  200       ['input_1[0][0]']             
                                                                                                  
 dense_15 (Dense)            (None, 100)                  200       ['input_1[0][0]']             
                                                                                                  
 dense_23 (Dense)            (None, 100)                  10100     ['dense_22[0][0]']            
                                                                                  

In [5]:
def clone_model(model):
    new_model = tf.keras.models.clone_model(model)
    new_model.compile(optimizer=model.optimizer, loss=model.loss)
    new_model.set_weights(model.get_weights())
    return new_model


In [6]:

List = []  # List to hold model copies

with tf.device('/GPU:0'):
    num_batch = 256
    epoch_size = 10
    for i in range(n-1, 0, -1):
        # Clone and save the current model state before re-training
        model_copy = clone_model(model)
        List.append(model_copy)

        concatenated_outputs = tf.concat([beta_dt * Y[:, i+1:i+2], dW[:, i:i+1] , Lw[:,i:i+1]], axis=1)
        model.fit(S[:, i:i+1], concatenated_outputs, epochs=epoch_size, batch_size=num_batch, validation_split=0.1, verbose=0)
        outputs = model.predict(S[:, i:i+1], verbose=0)
        psi_S = outputs[:, 1]
        phi_S = outputs[:, 0]
        psi_S_2= outputs[:,2]
        gamma_1 = outputs[:,3]
        gamma_2 = outputs[:,4]

        #to be faster :
        dW_true = dW[:, i]
        dW_true_2 = tf.square(dW_true)
        term_4 =  dW_true*((1/3)*dW_true_2 - dt)

        X[:, i].assign(beta_dt * X[:, i+1] - psi_S * dW_true-psi_S_2*(dW_true_2 - dt)-gamma_1 * Lw[:,i]-gamma_2*term_4)
        Y[:, i].assign(beta_dt * Y[:, i+1] - psi_S * dW_true-psi_S_2*(dW_true_2 - dt)-gamma_1 * Lw[:,i]-gamma_2*term_4)

        Z = pay_off(S[:, i], r, K, dt)
        Y[:, i].assign(tf.where(Z > phi_S, Z, Y[:, i]))
        X[:, i].assign(tf.where(Z > X[:, i], Z, X[:, i]))

        if i % 10 == 0:
            print("We are currently",1-i/50,"complete")
    i = 0
    concatenated_outputs = tf.concat([beta_dt * Y[:, i+1:i+2], dW[:, i:i+1] , Lw[:, i:i+1]], axis=1)
    model.fit(S[:, i:i+1], concatenated_outputs, epochs=epoch_size, batch_size=num_batch, validation_split=0.1, verbose=0)
    outputs = model.predict(S[:, i:i+1])

    psi_S = outputs[:, 1]
    phi_S = outputs[:, 0]
    psi_S_2= outputs[:, 2]
    gamma_1 = outputs[:,3]
    gamma_2 = outputs[:,4]

    #to be faster :
    dW_true = dW[:, i]
    dW_true_2 = tf.square(dW_true)
    term_4 =  dW_true*((1/3)*dW_true_2 - dt)

    X[:, i].assign(beta_dt * X[:, i+1] - psi_S * dW_true-psi_S_2*(dW_true_2 - dt)-gamma_1 * Lw[:,i]-gamma_2*term_4)
    Y[:, i].assign(beta_dt * Y[:, i+1] - psi_S * dW_true-psi_S_2*(dW_true_2 - dt)-gamma_1 * Lw[:,i]-gamma_2*term_4)

    final_model_copy = clone_model(model)
    List.append(final_model_copy)


We are currently 0.19999999999999996 complete
We are currently 0.4 complete
We are currently 0.6 complete
We are currently 0.8 complete
32/32 [==============================] - 0s 3ms/step


In [7]:
u0=np.mean(X[:,0])
l0=np.mean(Y[:,0])
print(u0)
print(l0)
print(np.abs(u0-l0))

5.353932
4.277478
1.0764537
